In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext('local')
sql = SQLContext(sc)

In [2]:
df = sql.read.format('json').load('reviews_Kindle_Store.json')

In [8]:
meta = sql.read.format('json').load('meta_Kindle_Store.json')

In [47]:
from splearn.preprocessing import SparkLabelEncoder
from splearn import BlockRDD

data = ["paris", "paris", "tokyo", "amsterdam"]
y = BlockRDD(sc.parallelize(data))

le = SparkLabelEncoder()
le.fit(y)
test = ["tokyo", "tokyo", "paris"]
y_test = BlockRDD(sc.parallelize(test))

le.transform(y_test).toarray()

test = [2, 2, 1]
y_test = BlockRDD(sc.parallelize(test))
le.inverse_transform(y_test).toarray()

array(['tokyo', 'tokyo', 'paris'], 
      dtype='|S9')

In [23]:
tj = df.join(meta,on='asin',how='inner')

In [24]:
text = tj.select(concat(col("summary"), lit(" "), col("reviewText"), lit(" "), col("description")))

In [33]:
text = text.toDF('sentences')

In [44]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sentences", outputCol="words")
words = tokenizer.transform(text)

In [43]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
remover.transform(text).show(truncate=False)

IllegalArgumentException: u'Field "words" does not exist.'

In [46]:
sentenceData = sql.createDataFrame([
    (0, ["I", "saw", "the", "red", "balloon"]),
    (1, ["Mary", "had", "a", "little", "lamb"])
], ["id", "raw"])

remover = StopWordsRemover(inputCol="raw", outputCol="filtered")
remover.transform(sentenceData).show(truncate=False)

+---+----------------------------+--------------------+
|id |raw                         |filtered            |
+---+----------------------------+--------------------+
|0  |[I, saw, the, red, balloon] |[saw, red, balloon] |
|1  |[Mary, had, a, little, lamb]|[Mary, little, lamb]|
+---+----------------------------+--------------------+



In [ ]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

sentenceDataFrame = sql.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

regexTokenizer = RegexTokenizer(inputCol="sentence", outputCol="words", pattern="\\W")
# alternatively, pattern="\\w+", gaps(False)

countTokens = udf(lambda words: len(words), IntegerType())

tokenized = tokenizer.transform(sentenceDataFrame)
tokenized.select("sentence", "words")\
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

regexTokenized = regexTokenizer.transform(sentenceDataFrame)
regexTokenized.select("sentence", "words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

In [ ]:
sf = graphlab.SFrame({'x': [1,2,3], 'y': ['fish', 'chips', 'salad']})
df = sf.to_spark_dataframe(sc, sql)
df.show()

In [ ]:
sql.createDataFrame(rdd)

In [ ]:
sframe = graphlab.SFrame.from_rdd(df, sc)

In [ ]:
rdd = sc.parallelize([(x,str(x), "hello") for x in range(0,5)])
df = sql.createDataFrame(rdd)
sframe = graphlab.SFrame.from_rdd(df, sc)
print sframe